In [1]:
!pip install -q pogema==1.2.2
!pip install -q stable_baselines3==2.1.0
!pip install -q sb3-contrib==2.1.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 69.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow 2.14.0 requires numpy>=1.23.5, but you have numpy 1.23.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 611.2 kB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
import gymnasium as gym
from pogema import GridConfig

from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy

In [3]:
grid_config = GridConfig(
    size=8,
    density=0.3,
    num_agents=1,
    max_episode_steps=30
)

env = gym.make("Pogema-v0",grid_config=grid_config)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you 

In [4]:
recurrent_ppo = RecurrentPPO(
    "MlpLstmPolicy",
    env,
    verbose=1,
    gamma=0.99,
    batch_size=128,
    learning_rate=4e-3,
    policy_kwargs=dict(net_arch=[256, 256]),
    seed=42

)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [6]:
val_env = gym.make("Pogema-v0",grid_config=grid_config)

val_recurrent_ppo = RecurrentPPO(
     "MlpLstmPolicy",
    val_env,
    verbose=1,
    gamma=0.99,
    batch_size=128,
    learning_rate=4e-3,
    policy_kwargs=dict(net_arch=[256, 256]),
    seed=42,


)

mean_reward, std_reward = evaluate_policy(
    val_recurrent_ppo,
    val_recurrent_ppo.get_env(),
    deterministic=True,
    n_eval_episodes=20,
)

print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
mean_reward: 0.10 +/- 0.30


In [7]:
# Train agent and save it
recurrent_ppo.learn(total_timesteps=int(1.2e5))
recurrent_ppo.save("saved/recurrentppo_baseline")

流式输出内容被截断，只能显示最后 5000 行内容。
|    value_loss           | 0.00108   |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 28        |
|    ep_rew_mean          | 0.07      |
| time/                   |           |
|    fps                  | 28        |
|    iterations           | 701       |
|    time_elapsed         | 3126      |
|    total_timesteps      | 89728     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -1.29e-12 |
|    explained_variance   | 0.862     |
|    learning_rate        | 0.004     |
|    loss                 | 6e-06     |
|    n_updates            | 7000      |
|    policy_gradient_loss | -6.75e-10 |
|    value_loss           | 1.56e-05  |
---------------------------------------
---------------------------------------
| rollout/   

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'saved' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [15]:
def evaluate_success_rate(model, env, num_episodes=10):
    success_count = 0
    step_array = []
    for _ in range(num_episodes):
        obs = env.reset()

        # Check if observation is a tuple and extract the first element if true.
        if isinstance(obs, tuple):
            obs = obs[0]
        max_step = 100
        steps_taken = 0
        done = truncated = False
        while not done and max_step > 0:
            action, _ = model.predict(obs)
            next_obs, reward, done, truncated, info = env.step(action)
            print(action,max_step,success_count,done)
            max_step -= 1
            steps_taken += 1
            # Check if next_obs is a tuple and extract the first element if true.
            if isinstance(next_obs, tuple):
                next_obs = next_obs[0]
            obs = next_obs

            # Check if agent was successful in that episode.
            if done:
                success_count += 1
                step_array.append(steps_taken)
                break

    success_rate = success_count / num_episodes
    return success_rate, step_array

success_rate,step_array = evaluate_success_rate(recurrent_ppo, env)
print(f"Agent Success Rate: {success_rate * 100:.2f}%")
print(f"steps to termination : {step_array}")

2 100 0 False
2 99 0 False
2 98 0 False
2 97 0 False
2 96 0 False
2 95 0 False
2 94 0 False
2 93 0 False
2 92 0 False
2 91 0 False
2 90 0 False
2 89 0 False
2 88 0 False
2 87 0 False
2 86 0 False
2 85 0 False
2 84 0 False
2 83 0 False
2 82 0 False
2 81 0 False
2 80 0 False
2 79 0 False
2 78 0 False
2 77 0 False
2 76 0 False
2 75 0 False
2 74 0 False
2 73 0 False
2 72 0 False
2 71 0 False
2 70 0 False
2 69 0 False
2 68 0 False
2 67 0 False
2 66 0 False
2 65 0 False
2 64 0 False
2 63 0 False
2 62 0 False
2 61 0 False
2 60 0 False
2 59 0 False
2 58 0 False
2 57 0 False
2 56 0 False
2 55 0 False
2 54 0 False
2 53 0 False
2 52 0 False
2 51 0 False
2 50 0 False
2 49 0 False
2 48 0 False
2 47 0 False
2 46 0 False
2 45 0 False
2 44 0 False
2 43 0 False
2 42 0 False
2 41 0 False
2 40 0 False
2 39 0 False
2 38 0 False
2 37 0 False
2 36 0 False
2 35 0 False
2 34 0 False
2 33 0 False
2 32 0 False
2 31 0 False
2 30 0 False
2 29 0 False
2 28 0 False
2 27 0 False
2 26 0 False
2 25 0 False
2 24 0 Fals